In [1]:
library(Seurat)
library(SeuratDisk)
library(sva) # only if using combat

Attaching SeuratObject



Load the BALF COVID dataset, which is described here: https://doi.org/10.1038/s41591-020-0901-9.

This dataset contains 12 samples, each associated with "Healthy Control", "Moderate", or "Severe" COVID contexts. 

You can download the 12 scRNAseq .h5 files under the samples section here: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE145926
You can also download the metadata file: TODO

Once loaded, we begin with a basic preprocessing of each sample based on QC metrics. See https://satijalab.org/seurat/articles/pbmc3k_tutorial.html for details

In [51]:
expression.data.path<-'/data2/eric/Tensor-Revisions/data/COVID-19/'

In [78]:
# load the metadata
md <- read.table(paste0(expression.data.path, 'metadata.txt'), header = T, row.names = 'ID')
colnames(md) = c('Sample.ID', 'sample_new', 'Context', 'disease', 'hasnCoV', 'cluster', 'cell.type')

context.map = c('Healthy.Control', 'Moderate.Covid', 'Severe.Covid')
names(context.map) <- c('HC', 'M', 'S')
md['Context'] <- unname(context.map[md$Context])

balf.samples<-list()
for (filename in list.files(expression.data.path)){
    if (endsWith(filename, '.h5')){
        sample<-unlist(strsplit(filename, '_'))[[2]]
        
        # subset and format metadata
        md.sample<-md[md$Sample.ID == sample,]
        rownames(md.sample) <- unname(sapply(rownames(md.sample), 
                                           function(x) paste0(unlist(strsplit(x, '_'))[[1]], '-1')))
        # load the counts
        so <- Seurat::Read10X_h5(filename=paste0(expression.data.path, filename), unique.features=T)
        so <- so[, rownames(md.sample)]
                                           
        # preprocess
        so <- CreateSeuratObject(counts=so, project=sample, meta.data=md.sample[c('Sample.ID', 'Context', 'cell.type')], 
                      min.features=30, min.cells=3)
        # filter cells based on QC metrics
        so[["percent.mt"]] <- PercentageFeatureSet(so, pattern = "^MT-")
        so <- subset(so, subset = nFeature_RNA < 6000 & percent.mt < 10)
        
        balf.samples[[sample]] <- so                                   
    }
}

Next, we normalize the raw UMI counts. We recommend log(1+CPM) normalization, as this maintains non-negative counts and is the input for many communication scoring functions

In [79]:
balf.samples <- lapply(balf.samples, 
                    function(so) NormalizeData(so, normalization.method = "LogNormalize", scale.factor = 1e6))

Finally, we apply a batch correction. The goal here is to account for sample-to-sample technical variability. 

At this point, we diverge from the Python preprocessing tutorial in order to leverage Seurat's built-in batch correction functions. To decrease run time, we will use reciprocal PCA instead of CCA. See https://satijalab.org/seurat/articles/integration_introduction.html and Seurat's other integration vignettes for additional details. To apply Combat as in scanpy, see commented code further below.

Note, the final input matrices to Tensor-cell2cell must be non-negative. We will demonstrate workarounds to negative counts in the tensor building tutorial.

In [107]:
batch.var <- 'Sample.ID' # the batch variable in the metadata

In [178]:
# get the HVGs for each sample separately
balf.samples <- lapply(balf.samples, 
                      function(so) FindVariableFeatures(so, selection.method = "vst", nfeatures = 2000))
                       
# find the common HVGs across samples
integration.features <- SelectIntegrationFeatures(object.list = balf.samples)

                       
# # to use CCA instead of reciprocal PCA, follow lines 10-12, instead of lines 14-22
# # find the integration anchors
# integration.anchors <- FindIntegrationAnchors(object.list = balf.samples, 
#                                               anchor.features = integration.features)    
                       
# calculate PCA on each sample separately
balf.samples <- lapply(X = balf.samples, FUN = function(x) {
    x <- ScaleData(x, features = integration.features, verbose = F)
    x <- RunPCA(x, features = integration.features, verbose = F)
})

# find the integration anchors
integration.anchors <- FindIntegrationAnchors(object.list = balf.samples, 
                                              anchor.features = integration.features, reduction = "rpca")

# do the batch correction
balf.corrected <- IntegrateData(anchorset = integration.anchors)

In [190]:
# 2000 top variable features were already calculated

# get PCA to 100 PCs
balf.corrected <- ScaleData(balf.corrected, verbose = F)
balf.corrected <- RunPCA(balf.corrected, npcs = 100, verbose = F)

In [193]:
balf.corrected

An object of class Seurat 
24900 features across 63102 samples within 2 assays 
Active assay: integrated (2000 features, 2000 variable features)
 1 other assay present: RNA
 1 dimensional reduction calculated: pca

Do the batch correction using Combat. This most closely emulated the steps taken in the scanpy tutorial, though there are still differences.

In [99]:
# # merge the balf samples

# # note, this is less stringent than scanpy's concat method and will retain all features
# balf.corrected<-merge(x = balf.samples[[1]], y = balf.samples[2:length(balf.samples)], 
#                       add.cell.ids = names(balf.samples), merge.data = TRUE)


# # prepare batch covariate
# batch<-balf.corrected@meta.data[[batch.var]]
# names(batch)<-row.names(balf.corrected@meta.data)
# batch<-as.factor(batch)

# # do the batch correction
# com <- ComBat(dat=balf.corrected@assays$RNA@data, batch=batch)

# # store the batch corrected matrix in the scale.data slot
# balf.corrected@assays$RNA@scale.data<-com

# # get the top 2000 highly variable genes
# balf.corrected<-FindVariableFeatures(balf.corrected, nfeatures=2000)

# # get PCA to 100 PCs, calculated on batch corrected matrix
# RunPCA(balf.corrected, features=VariableFeatures(balf.corrected), npcs=100, 
#        assay = 'RNA', slot = 'scale.data')

Finally, we can convert this batch-correct Seurat object to an AnnData object using SeuratDisk (see https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html for details). The resultant h5ad file should contain the same information as the saved h5ad file in the Python tutorial at the end of tutorial 01A.

In [196]:
out.path = '/data3/hratch/c2c_general/'
SaveH5Seurat(balf.corrected, filename = paste0(out.path, 'Rbatch_corrected_balf_covid.h5Seurat'))
Convert(paste0(out.path, 'Rbatch_corrected_balf_covid.h5Seurat'), dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Adding data for integrated

Adding scale.data for integrated

Adding variable features for integrated

No feature-level metadata found for integrated

Adding cell embeddings for pca

Adding loadings for pca

No projected loadings for pca

Adding standard deviations for pca

No JackStraw data for pca

Validating h5Seurat file

Adding scale.data from integrated as X

Adding data from integrated as raw

Transfering meta.data to obs

Adding dimensional reduction information for pca

Adding feature loadings for pca



The .h5ad file, named below, can be read into scanpy using scanpy.read_h5ad(filename)

In [197]:
filename=paste0(out.path, 'Rbatch_corrected_balf_covid.h5ad')
print(filename)

[1] "/data3/hratch/c2c_general/Rbatch_corrected_balf_covid.h5ad"
